In [1]:
from transformers import BartTokenizer, TFBartForConditionalGeneration
import whisper
import gradio as gr
model = whisper.load_model("base.en")

def summarize(transcript):
    model_dir = "FT_model"
    tokenizer = BartTokenizer.from_pretrained("FT_model/tokenizer")
    model = TFBartForConditionalGeneration.from_pretrained(model_dir)

    input_ids = tokenizer(transcript, truncation=True, return_tensors="pt").input_ids

    gens = {
        "length_penalty": 2.0,
        "num_beams": 4,
        "no_repeat_ngram_size": 3,
        "min_length": 39,
        "max_length": 250
    }
    outputs = model.generate(input_ids, **gens)
    summ = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summ

def transcribe_sum(audio_file):
    result = model.transcribe(audio_file)
    with open("transcript.txt", "w") as file:
        file.write(result["text"])
    
    with open('transcript.txt', 'r', encoding='utf-8') as file:
        transcript = file.read()
    final = summarize(transcript)
    return final

def read_sum(text_file):
    with open(text_file, 'r', encoding='utf-8') as file:
        transcript = file.read()
    final = summarize(transcript)
    return final

c:\Users\serix\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with gr.Blocks() as demo:
    demo.title="Podscast Summarization"
    with gr.Tab("Upload audio"):
        gr.Interface(
        title='Podcast Summarization',
        fn=transcribe_sum,
        inputs=gr.File(label='Upload Audio File', file_types=["audio"]),
        outputs=gr.Textbox(lines=8, show_copy_button=True, label='Summarized Output'),
        submit_btn="Generate",
        clear_btn=None)
        
    with gr.Tab("Upload Text"):
        title="Podcast Summarization"
        gr.Interface(
        title='Podcast Summarization',
        fn=read_sum,
        inputs=gr.File(label='Upload Text File', file_types=["text"]),
        outputs=gr.Textbox(lines=8, show_copy_button=True, label='Summarized Output'),
        submit_btn="Generate",
        clear_btn=None)

    with gr.Tab("Input Text"):
        gr.Interface(
        title='Podcast Summarization',
        fn=summarize,
        inputs=gr.Textbox(lines=8, placeholder='Enter Text Here...', label='Input text'),
        outputs=gr.Textbox(lines=8, show_copy_button=True, label='Summarized Output'),
        submit_btn="Generate",
        clear_btn=None)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at FT_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
